<a href="https://colab.research.google.com/github/UniVR-DH/DBMS-course/blob/main/notebooks/lecture01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
conn = sqlite3.connect("lecture01.db")
cursor = conn.cursor()

In [2]:
cursor.execute('''
CREATE TABLE book (
    book_id INTEGER PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    publisher_name VARCHAR(100),
    publication_year INTEGER,
    language CHAR(2)
)
''')

cursor.execute('''
CREATE TABLE author (
    author_id INTEGER PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    nationality VARCHAR(50)
)
''')

cursor.execute('''
CREATE TABLE book_author (
    book_id INTEGER,
    author_id INTEGER,
    PRIMARY KEY (book_id, author_id),
    FOREIGN KEY (book_id) REFERENCES book(book_id),
    FOREIGN KEY (author_id) REFERENCES author(author_id)
)
''')

conn.commit()
print("Ready!")

Ready!


In [3]:
# Insert books
cursor.execute('''
INSERT INTO book (book_id, title, publisher_name, publication_year, language)
VALUES
(1, 'To Kill a Mockingbird', 'HarperCollins', 1960, 'EN'),
(2, '1984', 'Penguin Books', 1949, 'EN'),
(3, 'One Hundred Years of Solitude', 'Harper & Row', 1967, 'ES'),
(4, 'Il Nome della Rosa', 'Bompiani', 1980, 'IT'),
(5, 'Good Omens', 'Workman', 1990, 'EN')
''')

# Insert authors
cursor.execute('''
INSERT INTO author (author_id, first_name, last_name, nationality)
VALUES
(1, 'Harper', 'Lee', 'American'),
(2, 'George', 'Orwell', 'British'),
(3, 'Gabriel', 'García Márquez', 'Colombian'),
(4, 'Umberto', 'Eco', 'Italian'),
(5, 'Terry', 'Pratchett', 'British'),
(6, 'Neil', 'Gaiman', 'British')
''')

# Insert book-author relationships
cursor.execute('''
INSERT INTO book_author (book_id, author_id)
VALUES
(1, 1),
(2, 2),
(3, 3),
(4, 4),
(5, 5),
(5, 6),
(2, 6)
''')

conn.commit()
print("Ready!")

Ready!


In [4]:
# Query to get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all results
tables = cursor.fetchall()

# Print the table names
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
book
author
book_author


In [6]:
# Get the current directory
current_dir = Path.cwd()

# List all files in the current directory
print("Files in the current directory:")
for file_path in current_dir.iterdir():
    if file_path.is_file():
        print(file_path.name)

Files in the current directory:
lecture01.db


In [7]:
# Select all books
cursor.execute("SELECT * FROM book")

# Fetch all results
books = cursor.fetchall()

# Print the book information
print("List of books:")
for book in books:
    print(book)


List of books:
(1, 'To Kill a Mockingbird', 'HarperCollins', 1960, 'EN')
(2, '1984', 'Penguin Books', 1949, 'EN')
(3, 'One Hundred Years of Solitude', 'Harper & Row', 1967, 'ES')
(4, 'Il Nome della Rosa', 'Bompiani', 1980, 'IT')
(5, 'Good Omens', 'Workman', 1990, 'EN')


In [8]:
# Execute a query to select all books
cursor.execute("SELECT * FROM book")

# Fetch all results
books = cursor.fetchall()

# Print the books
print("List of all books:")
for book in books:
    print(f"ID: {book[0]}, Title: {book[1]}, Publisher: {book[2]}, Year: {book[3]}, Language: {book[4]}")

List of all books:
ID: 1, Title: To Kill a Mockingbird, Publisher: HarperCollins, Year: 1960, Language: EN
ID: 2, Title: 1984, Publisher: Penguin Books, Year: 1949, Language: EN
ID: 3, Title: One Hundred Years of Solitude, Publisher: Harper & Row, Year: 1967, Language: ES
ID: 4, Title: Il Nome della Rosa, Publisher: Bompiani, Year: 1980, Language: IT
ID: 5, Title: Good Omens, Publisher: Workman, Year: 1990, Language: EN


In [22]:
cursor.close()

In [23]:
from google.colab import files
files.download('lecture01.db')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>